In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO  
from IPython.display import Image  


In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.model_selection import RandomizedSearchCV


In [41]:
from sklearn.metrics import accuracy_score

In [3]:
logins = pd.read_csv('takehome_user_engagement.csv')

In [4]:
logins.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven day period

In [5]:
logins['time_stamp'].dtype

dtype('O')

In [6]:
logins['time_stamp'] = pd.to_datetime(logins['time_stamp'])

In [7]:
logins['date'] = logins['time_stamp'].dt.date

In [8]:
logins.head()

,time_stamp,user_id,visited,date
0,2014-04-22 03:53:30,1,1,2014-04-22
1,2013-11-15 03:45:04,2,1,2013-11-15
2,2013-11-29 03:45:04,2,1,2013-11-29
3,2013-12-09 03:45:04,2,1,2013-12-09
4,2013-12-25 03:45:04,2,1,2013-12-25


In [9]:
logins.set_index('time_stamp', inplace=True)

In [10]:
result = logins.groupby('user_id').resample('7D')['visited'].sum()

In [11]:
print(result)

user_id  time_stamp
1        2014-04-22    1
2        2013-11-15    1
         2013-11-22    0
         2013-11-29    1
         2013-12-06    1
                      ..
11996    2013-09-06    1
11997    2013-01-15    1
11998    2014-04-27    1
11999    2012-06-02    1
12000    2014-01-26    1
Name: visited, Length: 80874, dtype: int64


In [12]:
visited_df = pd.DataFrame({'visited': result})

In [13]:
visited_df.head()

visited
user_id time_stamp         
1       2014-04-22        1
2       2013-11-15        1
        2013-11-22        0
        2013-11-29        1
        2013-12-06        1

In [14]:
visited_df['adopted'] = visited_df['visited'] >= 3

In [15]:
visited_df

visited  adopted
user_id time_stamp                  
1       2014-04-22        1    False
2       2013-11-15        1    False
        2013-11-22        0    False
        2013-11-29        1    False
        2013-12-06        1    False
...                     ...      ...
11996   2013-09-06        1    False
11997   2013-01-15        1    False
11998   2014-04-27        1    False
11999   2012-06-02        1    False
12000   2014-01-26        1    False

[80874 rows x 2 columns]

In [18]:
visited_df = visited_df.reset_index()

In [19]:
visited_df.head()

,user_id,time_stamp,visited,adopted
0,1,2014-04-22,1,False
1,2,2013-11-15,1,False
2,2,2013-11-22,0,False
3,2,2013-11-29,1,False
4,2,2013-12-06,1,False


In [23]:
# Create a dictionary to store the user id and >3 status
user_status = {}

# Iterate through each row of df1
for index, row in visited_df.iterrows():
    user_id = row['user_id']
    greater_than_3 = row['adopted']
    
    # Check if the user id already exists in the dictionary
    if user_id in user_status:
        # If the current row has >3 True, set the status to True
        if greater_than_3:
            user_status[user_id] = True
    else:
        # If the user id doesn't exist, initialize the status
        user_status[user_id] = greater_than_3

# Create the second dataframe (df2) using the dictionary
visited_df = pd.DataFrame(list(user_status.items()), columns=['user_id', 'adopted'])

# Display the result
print(visited_df)


      user_id  adopted
0           1    False
1           2     True
2           3    False
3           4    False
4           5    False
...       ...      ...
8818    11996    False
8819    11997    False
8820    11998    False
8821    11999    False
8822    12000    False

[8823 rows x 2 columns]


In [24]:
#visited_df.to_csv('>=3', index=False)

In [25]:
users = pd.read_csv('takehome_users.csv', encoding='latin-1')
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [26]:
merged = users.merge(visited_df, left_on='object_id', right_on='user_id', how='left', suffixes=('_u', '_v'))

In [27]:
len(merged)

12000

In [28]:
merged.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,3.0,False
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,4.0,False
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,5.0,False


In [32]:
#identify which factors predict future user adoption.
#classification model since 

merged['creation_source'].unique()

array(['GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS',
       'SIGNUP_GOOGLE_AUTH'], dtype=object)

In [33]:
#don't use org_id
merged['org_id'].nunique()

417

In [34]:
# missing values 
missing = pd.concat([merged.isnull().sum(), 100 * merged.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

In [35]:
dropna = merged.dropna(subset = ['adopted','creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip'])
features = ['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip']
dropna['adopted'] = dropna['adopted'].astype(int)

# Make an explanatory variable called X
X= dropna[features]

y= dropna['adopted']

/var/folders/9z/l0mdtnzj2x9436mcqq197dtm0000gn/T/ipykernel_36014/3560787111.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropna['adopted'] = dropna['adopted'].astype(int)


In [36]:
# One-hot encode all features in X.
X = pd.get_dummies(X)

In [37]:
X

,opted_in_to_mailing_list,enabled_for_marketing_drip,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,0,1,0,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
11995,0,0,0,1,0,0,0
11996,0,0,0,0,0,0,1
11997,1,1,1,0,0,0,0
11998,0,0,0,0,1,0,0


In [38]:
# Call train_test_split on X, y. Make the test_size = 0.25, and random_state = 246
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 246)

In [39]:
# Step 2: Create the model
model = GradientBoostingClassifier()

# Step 3: Train the model
model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = model.predict(X_test)

In [42]:
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy of the Gradient Boosting Classifier: {:.2f}%".format(accuracy * 100))

Accuracy of the Gradient Boosting Classifier: 83.68%


In [46]:
feature_importance = model.feature_importances_

# Create a DataFrame to display feature importance
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print('The factors that predict adoption are:\n')
print(importance_df)

The factors that predict adoption are:

                              Feature  Importance
2        creation_source_GUEST_INVITE    0.441409
4   creation_source_PERSONAL_PROJECTS    0.254167
0            opted_in_to_mailing_list    0.149341
1          enabled_for_marketing_drip    0.070010
5              creation_source_SIGNUP    0.040507
3          creation_source_ORG_INVITE    0.024972
6  creation_source_SIGNUP_GOOGLE_AUTH    0.019594
